In [108]:
import pandas as pd
import numpy as np
import time


data =pd.read_csv("docword_kos.txt",skiprows=3,header=None,sep= " ",names=["docId","wordId","count"])
data_clean=data.groupby(["docId","wordId"])["count"].sum().unstack().reset_index().fillna(0).set_index("docId")
data_clean[data_clean>0]=1

#set of all transactions
Transactions=[]
for id in data_clean.index:
    ls=np.flatnonzero(data_clean.loc[id]).tolist()
    ls1=[item+1 for item in ls]
    ls2=set(ls1)
    Transactions.append(ls2)

#candidate-gen function
def candidate_gen(f):
    c_set=[]
    c=set()
    for f1 in f:
        for f2 in f :
            if (((f2.difference(f1)).union(f1.difference(f2))=={max(f1),max(f2)})&(max(f1)<max(f2))):
                c =f1.union({max(f2)})
                c_set.append(c)
               
                for i in c:
                    if c.difference({i}) not in f:
                        
                        if (c_set != []) & (c in c_set):
                           
                            c_set.remove(c)
    
    return c_set

#counts the frequency of a candidate set
def support_count_of_sets(input_set):
    count=0
    for t in Transactions:
        if input_set.issubset(t):
            count =count+1
    return count
    

# apriori function
def apriori(data_clean,min_sup,k):
    c1=[]
    f=[]
    no_of_transactions=max(data_clean.index)
    start_time=time.time()
    for item in data_clean.columns:
        c1.append(set([item]))
        
    for item in data_clean.columns:
        if ((data_clean[item].sum())/no_of_transactions)>=min_sup:
            f.append(set([item]))
    final_f=[]
    final_f=final_f+f
   
    
    for i in range(2,k+1):
        
        c_set=candidate_gen(f)
        
        temp=len(f)
        for item in c_set:
            
            if (  support_count_of_sets(item) /no_of_transactions)>=min_sup:
                f.append(item)                           #  now f contains k-1,k length frequent item sets
                             
        f=f[temp:]                                       # this removes k-1 length frequent item sets from f
        final_f=final_f+f                                # appends all k length frequent items to f_final
    end_time  =time.time() 
    print("F=",min_sup)
    print("K=",k)
    print("Runtime is :"+str(end_time-start_time)+" seconds")
    print("The Frequent sets are:")
    print(final_f)
    return final_f

apriori(data_clean,.1,5)

apriori(data_clean,.2,10)

apriori(data_clean,.4,10)

